# waypoint_square.py

In this algorithm we are given a square land that drone has to traverse. We traverse along the sides of the square in a zic- zac manner and add the point in our output file. 
Finally printing the output file.

### importing headers

In [ ]:
from __future__ import print_function
import math

### Custom Class for taking lat/lon points

POINT class takes two parameters x and y and save them as lon and lat variables of the class object.

In [ ]:
class POINT:
	def __init__(self,x,y):
		self.lat = x
		self.lon =y

## Functions used: 

### 1. calNewGeoLocationNE(initialPOINT, yNORTH, xEAST):

This function is used to calculate new GEO Point which is 'y' meters north and 'x' meters east of a Reference point.

Knowing the lat/long of reference point and y and x distance , it returns a POINT class object with new location lat/long value.

This function is an approximation therefore valid over small distances (1m to 1km). It is not valid when calculating points close to the earth poles.

In [1]:
def calNewGeoLocationNE(initialPOINT, yNORTH, xEAST):
    
	#Radius of earth
	earthRad=6378137.0

	#New point offset calculated in radian
	tempLAT = yNORTH/earthRad
	tempLON = xEAST/(earthRad*math.cos(math.pi*initialPOINT.lat/180))

	#Now calculate the new point in decimal degrees
	finalLAT = initialPOINT.lat + (tempLAT * 180/math.pi)
	finalLON = initialPOINT.lon + (tempLON * 180/math.pi)
	finalLOCATION = POINT(finalLAT,finalLON)
	return finalLOCATION

### 2. distanceBetweenTwoGeoPoints(locPOINT1, locPOINT2):

This function calulates the ground distance between two points.

This function is a approximation therefore valid for only short distance. 


In [2]:
def get_distance_metres(aLocation1, aLocation2):
	disLatitude = locPOINT2.lat - locPOINT1.lat
	disLongitude = locPOINT2.lon - locPOINT1.lon
	return math.sqrt((disLatitude*disLatitude) + (disLongitude*disLongitude)) * 1.113195e5

### 3. def generate_points(start_point,edge_size,seed_distance,polygon_hull)

This function calculates the waypoints in a spiral manner for plantation of seeds in a square field.

    Input:
		alocation: location of centre of square in lat/lon
		aSize: half the side of square
		seed_distance: distance between two waypoints or seed plantation distance
	Output:
		A txt file named:  waypoint_square.txt, stores the waypoints generated in a sequence.

#### Opening the output file for writing purpose then calculating the left most bottom corner point of  square and adding first point to file.

In [4]:
def generate_points(aLocation,aSize,seed_distance):

	f = open("waypoint_square.txt","w+")
	temp1 = calNewGeoLocationNE(aLocation, -aSize, -aSize)
	f.write(str(temp1.lat) + "," + str(temp1.lon) + '\n')

#### Here step size which is the number of jumps required to cover one side of the square = aSize/seed_distance.

#### Outer for loop moves us in horizontal direction i.e left - right.
Here we move only halfway as we have two inner loops which traverse from bottom-top and top-bottom.

In [ ]:
	for i in range (aSize/seed_distance):
		temp2 = temp1

#### Inner for loop 1. it moves us from bottom-top.
As we move from bottom to top we simultaniously add the points in our output file.

In [ ]:
		for j in range(2*aSize/seed_distance):
			newpoint = calNewGeoLocationNE(temp2, seed_distance, 0)
			temp2 = newpoint
			f.write(str(temp2.lat) + "," + str(temp2.lon) + '\n')

#### Right shift operation is done here.
Here we make a move to the right and add the point to file. 

In [ ]:
		shift1 = calNewGeoLocationNE(temp2, 0, seed_distance)
		temp2  = shift1
		f.write(str(temp2.lat) + "," + str(temp2.lon) + '\n')

#### Inner loop 2. here the movement is top-bottom
As we move from top to bottom we simultaniously add the points in our output file.

In [ ]:
		for j in range(2*aSize/seed_distance):
			newpoint = calNewGeoLocationNE(temp2, -seed_distance, 0)
			temp2 = newpoint
			f.write(str(temp2.lat) + "," + str(temp2.lon) + '\n')

#### Right shift operation is done here.
Here we make a move to the right and add the point in our output file.

In [ ]:
		shift2 = calNewGeoLocationNE(temp2, 0, seed_distance)
		temp1 = shift2
		f.write(str(temp1.lat) + "," + str(temp1.lon) + '\n')

#### This loop computes the last waypoints along the last boundary of the convex structure.

In the above loops we move up-right-down-right. If we continue to move this way we will miss the last boundary of our square. So this loop does that for us.                           

In [ ]:
	for j in range(2*aSize/seed_distance):
			newpoint = calNewGeoLocationNE(temp1, seed_distance, 0)
			temp1 = newpoint
			f.write(str(temp1.lat) + "," + str(temp1.lon) + '\n')

## Main Body: 

#### Taking user input for land info:
User enters the latitude and longitude of the center of the land under consideration. We further check if the values entered is valid or not.

In [ ]:
print("   This code generates the required waypoint for the drone \n")
print("Please enter the geo location of the centre of your field: \n")
in_lat = 0.0
in_lon = 0.0
while True:
	try:
		in_lat = float(input("Please enter the latitute of centre:\n"))
		if(in_lat<0 or in_lat>90):
			print("Latitude value must be between 0 and 90")
			continue
		in_lon = float(input("Please enter the longitude of centre:\n"))
		if(in_lon<0 or in_lon>180):
			print("Langitude value must be between 0 and 180")
			continue
		break
	except:
		print("Oops!  That was no valid lat/lon.  Try again...")

Create out custom POINT object for input:

In [ ]:
initial_location = POINT(in_lat,in_lon)

### Taking user input for size of land:

User enters distance between the center and boundary of square land. 
We further check if the values entered is valid or not.


In [ ]:
side_size = 0
while True:
	try:
		side_size = int(input("Please enter the distance between the center and field edge (i.e. side/2):\n"))
		break
	except:
		print("Oops!  Please insert positive interger value.  Try again...")

#### Taking the seed distance as an input from the user
This distance plays a significant role while computing the waypoints. We also check if the distance is valid or not and display appropriate messages.

In [ ]:
distance = 0
while True:
	try:
		distance = int(input("Please enter the distance between two waypoints:\n"))
		break
	except:
		print("Oops!  That was no valid distance.  Try again...")

#### Calling the required function.

In [ ]:
generate_points(initial_location,side_size,distance)
print("\n   Waypoints are generated and stored in waypoint_square.txt file. \n")